In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torchvision.models import inception_v3
from torchvision.transforms import functional as TF
from scipy.linalg import sqrtm
from PIL import Image
from scipy.stats import entropy

# Function to Image Preprocessing
def preprocess_image(image_path, target_size=(299, 299)):
    image = Image.open(image_path)
    image = TF.resize(image, target_size)
    image = TF.to_tensor(image)
    if image.shape[0] == 1:  # If the image has a single channel (grayscale), convert it to RGB
        image = image.expand(3, -1, -1)
    image = TF.normalize(image, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Assuming normalization range is [-1, 1]
    return image.unsqueeze(0)  # Add a batch dimension

# Function to extract features from images using Inception-v3
def extract_features(images):
    inception_model = inception_v3(pretrained=True, progress=True, aux_logits=True)
    inception_model.eval()
    inception_model.to('cuda')  # Move the model to GPU if available
    features = []
    with torch.no_grad():
        images = images.to('cuda') if torch.cuda.is_available() else images
        features.append(inception_model(images)[0])
    return torch.stack(features)


# Function to calculate FID
def calculate_fid(real_images, generated_images):
    mu_real = torch.mean(real_images, dim=0)
    mu_generated = torch.mean(generated_images, dim=0)
    sigma_real = torch_cov(real_images)
    sigma_generated = torch_cov(generated_images)

    # Convert covariance matrices to numpy arrays
    sigma_real_np = sigma_real.cpu().numpy()
    sigma_generated_np = sigma_generated.cpu().numpy()

    # Calculate FID
    fid = torch.norm(mu_real - mu_generated) + torch.trace(torch.tensor(sqrtm(sigma_real_np @ sigma_generated_np)))
    return fid.item()




def torch_cov(m, rowvar=False):
    if m.dim() < 2:
        raise ValueError('m has less than 2 dimensions')

    # Ensure that 'rowvar' is properly handled
    if rowvar:
        m = m.t()
    if m.size(0) == 1:
        return torch.zeros(m.size(1), m.size(1))  # Return tensor of zeros if there's only one sample

    # Compute the covariance matrix
    m -= torch.mean(m, dim=0, keepdim=True)
    factor = 1 / (m.size(0) - 1)
    cov_matrix = factor * m.t() @ m
    return cov_matrix.unsqueeze(0) if cov_matrix.dim() == 2 else cov_matrix




# Load and preprocess real and generated images
real_image_path = "/content/sample_data/cat1.jpg"
generated_image_path = "/content/sample_data/cat2.jpg"

real_image = preprocess_image(real_image_path)
generated_image = preprocess_image(generated_image_path)

# Extract features from images
real_features = extract_features(real_image)
generated_features = extract_features(generated_image)

# Calculate FID
fid_score = calculate_fid(real_features, generated_features)
print("FID score:", fid_score)


In [ ]:
from scipy.stats import entropy

def inception_score(generated_images, num_splits=10):
    inception_model = inception_v3(pretrained=True, progress=True, aux_logits=True)
    inception_model.eval()
    inception_model.to('cuda' if torch.cuda.is_available() else 'cpu')

    scores = []
    with torch.no_grad():
        for _ in range(num_splits):
            preds = []
            for i in range(0, generated_images.size(0), 32):
                batch = generated_images[i:i+32].to('cuda' if torch.cuda.is_available() else 'cpu')
                pred = F.softmax(inception_model(batch), dim=1).cpu().numpy()
                preds.append(pred)
            preds = np.concatenate(preds, axis=0)
            scores.append(entropy(np.mean(preds, axis=0), base=2))
    return np.mean(scores), np.std(scores)

# Assuming generated_images is a tensor of shape [batch_size, channels, height, width]

# Calculate Inception Score
is_mean, is_std = inception_score(generated_image)
print("Inception Score (mean):", is_mean)
print("Inception Score (std):", is_std)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Inception Score (mean): 2.390452045766854
Inception Score (std): 0.0
